In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movehub-city-rankings/movehubcostofliving.csv
/kaggle/input/movehub-city-rankings/cities.csv
/kaggle/input/movehub-city-rankings/movehubqualityoflife.csv


In [2]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt

In [10]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve, accuracy_score 
from sklearn import linear_model

In [11]:
import warnings 
import math

In [15]:
data_cidades = pd.read_csv('../input/movehub-city-rankings/cities.csv')
data_q_vida = pd.read_csv('../input/movehub-city-rankings/movehubqualityoflife.csv')
data_c_vida = pd.read_csv('../input/movehub-city-rankings/movehubcostofliving.csv')

In [16]:
data_cidades.head() 

,City,Country
0,Oakland,United States
1,Oakville,Canada
2,Oaxaca de Juárez,Mexico
3,Oberhausen,Germany
4,Obihiro,Japan


In [17]:
data_cidades.tail() 

,City,Country
3538,Austin,United States
3539,Avellaneda,Argentina
3540,Avignon,France
3541,Ayacucho,Peru
3542,Aydın,Turkey


In [18]:
data_cidades.shape

(3543, 2)

In [19]:
data_q_vida.head()

,City,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
0,Caracas,65.18,11.25,44.44,83.45,8.61,85.70
1,Johannesburg,84.08,53.99,59.98,47.39,51.26,83.93
2,Fortaleza,80.17,52.28,45.46,66.32,36.68,78.65
3,Saint Louis,85.25,80.40,77.29,31.33,87.51,78.13
4,Mexico City,75.07,24.28,61.76,18.95,27.91,77.86


In [20]:
data_q_vida.tail()

,City,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
211,Dusseldorf,83.45,72.52,75.98,8.61,88.02,15.34
212,Tokyo,80.00,47.57,71.53,30.54,69.29,13.91
213,Abu Dhabi,86.40,68.03,48.02,53.42,80.80,10.86
214,Marbella,81.96,60.14,83.64,32.27,75.24,9.11
215,Malaga,84.30,45.77,69.17,30.55,67.90,9.11


In [21]:
data_q_vida.shape

(216, 7)

In [22]:
data_c_vida.head()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income
0,Lausanne,3.15,12.59,8.40,1.32,1714.00,4266.11
1,Zurich,3.28,12.59,8.40,1.31,2378.61,4197.55
2,Geneva,2.80,12.94,10.49,1.28,2607.95,3917.72
3,Basel,3.50,11.89,7.35,1.25,1649.29,3847.76
4,Perth,2.87,11.43,10.08,0.97,2083.14,3358.55


In [23]:
data_c_vida.tail()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income
211,Davao,0.79,1.90,3.17,0.84,554.18,158.34
212,Karachi,1.00,3.27,5.11,0.67,197.78,139.60
213,Lahore,1.23,3.27,6.54,0.66,206.08,132.95
214,Addis Ababa,0.46,2.29,4.18,0.72,653.77,124.22
215,Indore,0.91,2.23,6.03,0.84,205.15,120.68


In [24]:
data_c_vida.shape

(216, 7)

In [25]:
data_cidades.isna().sum()

City       0
Country    3
dtype: int64

In [26]:
data_q_vida.isna().sum()

City               0
Movehub Rating     0
Purchase Power     0
Health Care        0
Pollution          0
Quality of Life    0
Crime Rating       0
dtype: int64

In [27]:
data_c_vida.isna().sum()

City                     0
Cappuccino               0
Cinema                   0
Wine                     0
Gasoline                 0
Avg Rent                 0
Avg Disposable Income    0
dtype: int64

In [28]:
data_cidades[data_cidades['Country'].isnull()]

,City,Country
654,Sevastopol,NaN
724,Simferopol,NaN
1529,Priština,NaN


In [34]:
data_cidades.iloc[654, 1]= 'Ukraine'

In [35]:
data_cidades.iloc[724, 1]= 'Russia'

In [36]:
data_cidades.iloc[1529, 1]= 'Kosovo'

In [37]:
data_cidades[data_cidades['Country'].isnull()]

,City,Country


In [39]:
data_q_c_vida = pd.merge(data_c_vida, data_q_vida)

In [40]:
data_q_c_vida.head()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
0,Lausanne,3.15,12.59,8.40,1.32,1714.00,4266.11,87.21,90.77,65.85,87.62,73.21,35.55
1,Zurich,3.28,12.59,8.40,1.31,2378.61,4197.55,85.89,79.58,78.33,18.04,97.91,15.35
2,Geneva,2.80,12.94,10.49,1.28,2607.95,3917.72,83.27,61.22,74.88,29.43,82.76,54.36
3,Basel,3.50,11.89,7.35,1.25,1649.29,3847.76,84.20,78.17,79.74,59.18,88.27,28.12
4,Perth,2.87,11.43,10.08,0.97,2083.14,3358.55,95.38,62.11,80.56,23.53,74.62,50.01


In [41]:
data_q_c_vida.tail()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
211,Davao,0.79,1.90,3.17,0.84,554.18,158.34,69.29,17.63,69.00,77.98,30.66,17.71
212,Karachi,1.00,3.27,5.11,0.67,197.78,139.60,67.81,14.63,51.57,69.63,18.50,74.36
213,Lahore,1.23,3.27,6.54,0.66,206.08,132.95,67.32,12.45,48.94,30.55,28.52,51.98
214,Addis Ababa,0.46,2.29,4.18,0.72,653.77,124.22,59.88,6.38,63.89,85.59,28.41,26.04
215,Indore,0.91,2.23,6.03,0.84,205.15,120.68,76.38,35.79,31.34,59.42,48.53,16.93


In [42]:
data_q_c_vida.shape

(216, 13)

In [43]:
data_q_c_vida.dtypes

City                      object
Cappuccino               float64
Cinema                   float64
Wine                     float64
Gasoline                 float64
Avg Rent                 float64
Avg Disposable Income    float64
Movehub Rating           float64
Purchase Power           float64
Health Care              float64
Pollution                float64
Quality of Life          float64
Crime Rating             float64
dtype: object

In [45]:
data_q_c_vida = data_q_c_vida.sort_values(by='City')

In [46]:
data_q_c_vida.reset_index(drop=True)

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
0,Aachen,2.05,6.88,4.26,1.33,767.23,1619.72,81.64,60.55,73.25,11.69,90.52,15.34
1,Aberdeen,1.99,6.98,5.98,1.37,1195.74,1743.78,81.89,49.70,82.86,34.31,76.77,24.22
2,Abu Dhabi,2.67,6.23,13.73,0.30,1779.93,2135.92,86.40,68.03,48.02,53.42,80.80,10.86
3,Addis Ababa,0.46,2.29,4.18,0.72,653.77,124.22,59.88,6.38,63.89,85.59,28.41,26.04
4,Adelaide,2.49,11.42,10.08,0.95,1382.26,2911.69,87.29,72.03,56.25,12.01,91.54,41.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Warsaw,1.66,5.19,5.19,1.18,726.59,664.31,76.76,35.77,63.33,86.16,51.82,32.03
212,Washington,2.37,7.85,7.85,0.69,2451.63,2484.32,81.74,56.61,64.44,30.55,76.92,51.43
213,Wellington,2.20,8.82,8.27,1.17,1515.65,1763.67,81.06,49.11,73.72,30.55,79.83,27.38
214,Zagreb,1.12,3.70,4.48,1.23,504.36,605.23,73.84,26.71,52.78,83.45,47.59,32.29


In [48]:
data_q_c_vida_2 = pd.merge(data_q_c_vida, data_cidades,how='left', on='City')

In [49]:
data_q_c_vida_2.head() 

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
0,Aachen,2.05,6.88,4.26,1.33,767.23,1619.72,81.64,60.55,73.25,11.69,90.52,15.34,Germany
1,Aberdeen,1.99,6.98,5.98,1.37,1195.74,1743.78,81.89,49.70,82.86,34.31,76.77,24.22,United Kingdom
2,Abu Dhabi,2.67,6.23,13.73,0.30,1779.93,2135.92,86.40,68.03,48.02,53.42,80.80,10.86,United Arab Emirates
3,Addis Ababa,0.46,2.29,4.18,0.72,653.77,124.22,59.88,6.38,63.89,85.59,28.41,26.04,Ethiopia
4,Adelaide,2.49,11.42,10.08,0.95,1382.26,2911.69,87.29,72.03,56.25,12.01,91.54,41.32,Australia


In [50]:
data_q_c_vida_2[data_q_c_vida_2['Country'].isnull()]

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
10,Asheville,1.80,5.88,7.85,0.64,1225.82,1898.54,83.17,70.51,79.02,30.55,81.34,30.21,NaN
51,Cebu,1.43,2.85,3.48,0.88,475.01,200.28,66.93,12.90,73.70,35.90,41.21,46.88,NaN
61,Cordoba,1.95,3.92,4.25,0.93,524.80,523.02,73.76,27.56,75.24,48.76,47.68,51.56,NaN
66,Davao,0.79,1.90,3.17,0.84,554.18,158.34,69.29,17.63,69.00,77.98,30.66,17.71,NaN
74,Dusseldorf,2.13,8.31,4.47,1.28,1193.48,2301.70,83.45,72.52,75.98,8.61,88.02,15.34,NaN
79,Frankfurt,2.13,7.67,4.07,1.36,1022.98,1875.46,82.75,62.60,65.33,14.54,88.04,27.55,NaN
81,Galway,2.13,7.67,6.39,1.39,852.48,1875.46,83.19,58.59,52.97,52.47,77.85,40.78,NaN
100,Istanbul,1.99,5.44,9.06,1.66,634.23,724.84,80.22,34.42,80.38,44.98,45.67,42.36,NaN
101,Izmir,1.81,4.71,7.25,1.69,543.63,590.57,76.07,32.12,57.33,30.55,42.90,25.91,NaN
122,Lodz,1.45,4.77,4.15,1.18,337.35,415.20,73.32,26.37,53.21,37.21,46.29,64.51,NaN


In [51]:
data_q_c_vida_2.iloc[227,0]='Zürich'

In [52]:
data_q_c_vida_2.iloc[224,0]='Washington, D.C.'

In [53]:
data_q_c_vida_2.iloc[201,0]='Tampa, Florida'

In [54]:
data_q_c_vida_2.iloc[188,0]='São Paulo'

In [56]:
data_q_c_vida_2.iloc[185,0]='San Francisco, California'

In [57]:
data_q_c_vida_2.iloc[184,0]='San Diego, California'

In [58]:
data_q_c_vida_2.iloc[193,13]='Malta'

In [61]:
data_q_c_vida_2.iloc[10,13]='United States'

In [62]:
data_q_c_vida_2.iloc[51,13]='Philippines'

In [64]:
data_q_c_vida_2.iloc[61,13]='Argentina' 

In [65]:
data_q_c_vida_2.iloc[66,0]='Davao City'

In [67]:
data_q_c_vida_2.iloc[74,0]='Düsseldorf'

In [69]:
data_q_c_vida_2.iloc[79,0]='Frankfurt am Main'

In [71]:
data_q_c_vida_2.iloc[81,13]='Ireland' 

In [72]:
data_q_c_vida_2.iloc[100,0]='İstanbul'

In [74]:
data_q_c_vida_2.iloc[101,0]='İzmir'


In [76]:
data_q_c_vida_2.iloc[122,13]='Poland'


In [77]:
data_q_c_vida_2.iloc[129,0]='Málaga'


In [78]:
data_q_c_vida_2.iloc[130,0]='Malmö'


In [79]:
data_q_c_vida_2.iloc[134,13]='Spain'


In [80]:
data_q_c_vida_2.iloc[136,0]='Medellín'


In [81]:
data_q_c_vida_2.iloc[139,0]='Miami, Florida'


In [82]:
data_q_c_vida_2.iloc[141,0]='Minneapolis, Minnesota'


In [84]:
data_q_c_vida_2.iloc[164,13]='Thailand'


In [85]:
data_q_c_vida_2.iloc[166,0]='Philadelphia, Pennsylvania'


In [86]:
data_q_c_vida_2.iloc[167,0]='Phoenix, Arizona'


In [88]:
data_q_c_vida_2.iloc[168,0]='Portland, Oregon'


In [89]:
data_q_c_vida_2.iloc[176,0]='Rio de Janeiro'

In [90]:
data_q_c_vida_2.iloc[178,13]='United States'

In [91]:
data_q_c_vida_2.iloc[183,0]='San Antonio, Texas'

In [93]:
data= pd.merge(data_q_c_vida_2, data_cidades, how='inner', on='City')

In [94]:
data.head()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country_x,Country_y
0,Aachen,2.05,6.88,4.26,1.33,767.23,1619.72,81.64,60.55,73.25,11.69,90.52,15.34,Germany,Germany
1,Aberdeen,1.99,6.98,5.98,1.37,1195.74,1743.78,81.89,49.70,82.86,34.31,76.77,24.22,United Kingdom,United Kingdom
2,Abu Dhabi,2.67,6.23,13.73,0.30,1779.93,2135.92,86.40,68.03,48.02,53.42,80.80,10.86,United Arab Emirates,United Arab Emirates
3,Addis Ababa,0.46,2.29,4.18,0.72,653.77,124.22,59.88,6.38,63.89,85.59,28.41,26.04,Ethiopia,Ethiopia
4,Adelaide,2.49,11.42,10.08,0.95,1382.26,2911.69,87.29,72.03,56.25,12.01,91.54,41.32,Australia,Australia


In [95]:
data.tail() 

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country_x,Country_y
242,Warsaw,1.66,5.19,5.19,1.18,726.59,664.31,76.76,35.77,63.33,86.16,51.82,32.03,Poland,Poland
243,"Washington, D.C.",2.37,7.85,7.85,0.69,2451.63,2484.32,81.74,56.61,64.44,30.55,76.92,51.43,NaN,United States
244,Wellington,2.20,8.82,8.27,1.17,1515.65,1763.67,81.06,49.11,73.72,30.55,79.83,27.38,New Zealand,New Zealand
245,Zagreb,1.12,3.70,4.48,1.23,504.36,605.23,73.84,26.71,52.78,83.45,47.59,32.29,Croatia,Croatia
246,Zürich,3.28,12.59,8.40,1.31,2378.61,4197.55,85.89,79.58,78.33,18.04,97.91,15.35,NaN,Switzerland


In [96]:
data= data.drop('Country_x', axis=1) #dropando cloluna com valores nulos
data=data.rename(columns={'Country_y': 'Country'}) #renomeando coluna que vamos trabalhar

data[data['Country'].isnull()] 

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country


In [97]:
data.isna().sum()


City                     0
Cappuccino               0
Cinema                   0
Wine                     0
Gasoline                 0
Avg Rent                 0
Avg Disposable Income    0
Movehub Rating           0
Purchase Power           0
Health Care              0
Pollution                0
Quality of Life          0
Crime Rating             0
Country                  0
dtype: int64

In [99]:
cidades_duplicadas = data.City.value_counts()

In [100]:
cidades_duplicadas.head(11)

Cambridge     9
Valencia      9
Hamilton      4
Colombo       4
Vancouver     4
Athens        4
Birmingham    4
Barcelona     4
Hyderabad     4
London        4
Noida         1
Name: City, dtype: int64

In [101]:
data[(data['City'] == 'Valencia')]

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
224,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Spain
225,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Philippines
226,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Venezuela
227,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Spain
228,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Philippines
229,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Venezuela
230,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Spain
231,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Philippines
232,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Venezuela


In [102]:
data[(data['City'] == 'Cambridge')]

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
49,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United States
50,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,Canada
51,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United Kingdom
52,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United States
53,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,Canada
54,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United Kingdom
55,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United States
56,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,Canada
57,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United Kingdom


In [103]:
data[(data['City'] == 'London')]


,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
136,London,2.49,11.26,6.98,1.4,2590.76,1992.89,100.0,53.17,69.53,57.95,47.89,50.14,United Kingdom
137,London,2.49,11.26,6.98,1.4,2590.76,1992.89,100.0,53.17,69.53,57.95,47.89,50.14,Canada
138,London,2.49,11.26,6.98,1.4,2590.76,1992.89,100.0,53.17,69.53,57.95,47.89,50.14,United Kingdom
139,London,2.49,11.26,6.98,1.4,2590.76,1992.89,100.0,53.17,69.53,57.95,47.89,50.14,Canada


In [109]:
data.set_index('City').index.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True, False, False, False, False,
       False, False, False, False,  True,  True,  True, False, False,
       False, False, False, False, False, False,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [110]:
data = data.drop_duplicates(subset=['City', 'Country'])

In [112]:
cidades_duplicadas = data.City.value_counts()

In [113]:
cidades_duplicadas.head(11)

Cambridge     3
Valencia      3
Hamilton      2
Colombo       2
Vancouver     2
Athens        2
Birmingham    2
Barcelona     2
Hyderabad     2
London        2
Noida         1
Name: City, dtype: int64

In [114]:
data[(data['City'] == 'Valencia')]

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
224,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Spain
225,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Philippines
226,Valencia,1.28,6.82,4.26,1.24,596.74,1193.48,81.06,49.11,72.07,35.24,64.89,26.04,Venezuela


In [115]:
data[(data['City'] == 'Cambridge')] 

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
49,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United States
50,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,Canada
51,Cambridge,1.99,8.22,7.97,1.39,1345.2,2730.26,82.15,54.76,81.48,57.18,70.61,24.22,United Kingdom


In [116]:
data[(data['City'] == 'London')] 

,City,Cappuccino,Cinema,Wine,Gasoline,Avg Rent,Avg Disposable Income,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Country
136,London,2.49,11.26,6.98,1.4,2590.76,1992.89,100.0,53.17,69.53,57.95,47.89,50.14,United Kingdom
137,London,2.49,11.26,6.98,1.4,2590.76,1992.89,100.0,53.17,69.53,57.95,47.89,50.14,Canada


In [118]:
nomes_cidades = data.City.value_counts()

In [119]:
nomes_cidades[nomes_cidades> 1] 

Cambridge     3
Valencia      3
Hamilton      2
Colombo       2
Vancouver     2
Athens        2
Birmingham    2
Barcelona     2
Hyderabad     2
London        2
Name: City, dtype: int64

In [120]:
data.columns = data.columns.str.replace(' ','_')

In [121]:
data.head()

,City,Cappuccino,Cinema,Wine,Gasoline,Avg_Rent,Avg_Disposable_Income,Movehub_Rating,Purchase_Power,Health_Care,Pollution,Quality_of_Life,Crime_Rating,Country
0,Aachen,2.05,6.88,4.26,1.33,767.23,1619.72,81.64,60.55,73.25,11.69,90.52,15.34,Germany
1,Aberdeen,1.99,6.98,5.98,1.37,1195.74,1743.78,81.89,49.70,82.86,34.31,76.77,24.22,United Kingdom
2,Abu Dhabi,2.67,6.23,13.73,0.30,1779.93,2135.92,86.40,68.03,48.02,53.42,80.80,10.86,United Arab Emirates
3,Addis Ababa,0.46,2.29,4.18,0.72,653.77,124.22,59.88,6.38,63.89,85.59,28.41,26.04,Ethiopia
4,Adelaide,2.49,11.42,10.08,0.95,1382.26,2911.69,87.29,72.03,56.25,12.01,91.54,41.32,Australia


In [122]:
data.describe()

,Cappuccino,Cinema,Wine,Gasoline,Avg_Rent,Avg_Disposable_Income,Movehub_Rating,Purchase_Power,Health_Care,Pollution,Quality_of_Life,Crime_Rating
count,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000,219.000000
mean,1.986667,6.834155,7.113470,1.011963,1105.034384,1427.846027,79.896119,46.546164,66.517443,45.833333,59.865434,41.268995
std,0.742531,5.604582,3.305689,0.352639,670.672649,910.889913,6.508671,20.210330,14.358674,25.616140,22.006483,16.210121
min,0.460000,1.810000,2.130000,0.070000,120.680000,120.680000,59.880000,6.380000,20.830000,0.000000,5.290000,9.110000
25%,1.315000,4.585000,4.260000,0.770000,607.890000,588.790000,75.375000,29.135000,59.635000,24.280000,42.735000,28.805000
50%,2.090000,6.820000,6.540000,0.980000,980.650000,1554.460000,81.060000,49.110000,67.780000,37.270000,64.890000,40.960000
75%,2.490000,7.970000,8.430000,1.330000,1419.290000,2058.720000,84.040000,61.175000,77.235000,67.995000,78.565000,51.010000
max,4.480000,79.490000,26.150000,1.690000,5052.310000,4266.110000,100.000000,91.850000,95.960000,92.420000,97.910000,85.700000


In [126]:
col = data.drop(['City', 'Country'], axis = 1)

In [127]:
col[(np.abs(col)> 3000).any(1)]

,Cappuccino,Cinema,Wine,Gasoline,Avg_Rent,Avg_Disposable_Income,Movehub_Rating,Purchase_Power,Health_Care,Pollution,Quality_of_Life,Crime_Rating
25,3.50,11.89,7.35,1.25,1649.29,3847.76,84.20,78.17,79.74,59.18,88.27,28.12
30,3.92,12.32,12.32,1.57,1725.37,3002.59,80.74,51.24,84.95,18.40,83.76,16.67
58,2.35,11.42,10.08,0.99,1984.74,3023.91,83.23,63.26,91.90,11.48,93.05,40.36
92,2.80,12.94,10.49,1.28,2607.95,3917.72,83.27,61.22,74.88,29.43,82.76,54.36
105,2.78,5.89,10.10,1.52,5052.31,2210.39,86.37,50.07,67.59,14.88,59.50,16.31
128,3.15,12.59,8.40,1.32,1714.00,4266.11,87.21,90.77,65.85,87.62,73.21,35.55
165,3.84,12.00,13.50,0.65,2257.14,3089.75,80.61,80.30,60.30,0.00,80.50,25.50
167,2.58,8.50,9.81,0.69,3268.84,2530.09,90.53,55.56,51.39,12.01,61.94,47.33
178,2.87,11.43,10.08,0.97,2083.14,3358.55,95.38,62.11,80.56,23.53,74.62,50.01
204,2.64,5.27,15.82,1.11,3164.42,1582.21,81.07,33.14,58.82,18.48,42.65,19.96


In [128]:
col[(np.abs(col)< 0).any(1)]

,Cappuccino,Cinema,Wine,Gasoline,Avg_Rent,Avg_Disposable_Income,Movehub_Rating,Purchase_Power,Health_Care,Pollution,Quality_of_Life,Crime_Rating


In [129]:
superior = 3000
avg = 1427

data.loc[(data['Avg_Disposable_Income']>superior), 'Avg_Disposable_Income'] = avg
data.loc[(data['Avg_Rent']>superior), 'Avg_Rent'] = avg

In [131]:
X = data.drop(['Movehub_Rating','Country','City'], axis = 1) 
Y = data['Movehub_Rating']
test_size=0.30
seed=42

In [132]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X=StandardScaler().fit_transform(X)
scaler

StandardScaler()

In [133]:
from sklearn.model_selection import train_test_split

# utilização train_test_split, 30% teste e 70% treino

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [134]:
reg_log = linear_model.LinearRegression()
reg_log.fit (X_train,Y_train)

Y_pred = reg_log.predict(X_test)

In [135]:
from sklearn.metrics import explained_variance_score
explained_variance_score=explained_variance_score(Y_test, Y_pred) 

In [136]:
from sklearn.metrics import mean_squared_error
mean_squared_error=mean_squared_error(Y_test, Y_pred)

In [137]:
from sklearn.metrics import mean_absolute_error
erro_medio_absoluto=mean_absolute_error(Y_test, Y_pred)

In [138]:
print ("Determination coefficient r²: {:.3f}".format(reg_log.score(X_train,Y_train)))
print ("Determination coefficient r² of the testing set: {:.3f} ".format(reg_log.score(X_test,Y_test)))
print("mean_squared_error: {:.2f} ".format(mean_squared_error))
print("mean_absolute_error: {:.2f} ".format(erro_medio_absoluto))
print("Explained_variance_score: {:.3f} ".format(explained_variance_score))
print ('Linear Coefficient {:.2f}'.format(reg_log.intercept_))
print ('Angular Coefficient{:}'.format(reg_log.coef_))

Determination coefficient r²: 0.734
Determination coefficient r² of the testing set: 0.743 
mean_squared_error: 10.68 
mean_absolute_error: 2.17 
Explained_variance_score: 0.745 
Linear Coefficient 79.83
Angular Coefficient[-0.73182651  0.23531748  0.30737501  1.30599598  1.24213927  1.04085506
  4.20873134  0.37326308 -0.26249257 -0.33492937  0.39987909]


In [139]:
pd.DataFrame(list(zip(Y_test[10:20],Y_pred[10:20])),columns=['Avaliação Movehub','Predição']) 

,Avaliação Movehub,Predição
0,76.07,77.111299
1,70.60,71.397373
2,82.48,84.929642
3,71.91,74.258543
4,90.73,87.310223
5,74.60,74.433542
6,82.43,82.079814
7,74.21,75.001270
8,80.94,82.632363
9,71.58,71.119579
